In [5]:
import functools
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchtext
import tqdm

print(sys.executable)

C:\Users\Tyrone\anaconda3\envs\nlp\python.exe


In [6]:
train_iter = torchtext.datasets.IMDB(split='train')

In [8]:
tokenizor = torchtext.data.utils.get_tokenizer('basic_english')

In [9]:
tokens = []
for label, line in train_iter:
    tokens += tokenizor(line)

In [10]:
from collections import Counter, OrderedDict
# from torchtext.vocab import Vocab

counter = Counter(tokens) # unsorted dict
sorted_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_tuples)
Vocab = torchtext.vocab.vocab(ordered_dict, min_freq=10)

# adding <unk> token and default index
unk_token = '<unk>'
default_index = 0
if unk_token not in Vocab:
    Vocab.insert_token(unk_token, default_index)
Vocab.set_default_index(default_index)

# adding <pad> token
Vocab.insert_token('<pad>', 1)

In [23]:
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import random

text_transform = lambda x: [Vocab[token] for token in tokenizor(x)]
label_transform = lambda x: 1 if x == 'pos' else 0

def collate_batch(batch):
    label_list, text_list = [], []
    for label, text in batch:
        text_list.append(torch.tensor(text_transform(text)))
        label_list.append(label_transform(label))
    
    return torch.tensor(label_list), pad_sequence(text_list, padding_value=Vocab['<pad>'])

train_iter, test_iter = torchtext.datasets.IMDB(split=('train', 'test'))

class dataset_from_iter(Dataset):
    def __init__(self, train_iter):
        self.labels, self.text = [], []
        for label, line in train_iter:
            self.labels.append(label)
            self.text.append(line)
    
    def __getitem__(self, index):
        return self.labels[index], self.text[index]
    
    def __len__(self):
        return len(self.labels)

train_dataset = dataset_from_iter(train_iter)

n_train = len(train_dataset)
split = n_train // 3
indices = list(range(n_train))
random.shuffle(indices)
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(indices[split:])
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(indices[:split])

train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=train_sampler, collate_fn=collate_batch)
valid_dataloader = DataLoader(train_dataset, batch_size=8, sampler=valid_sampler, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter, batch_size=8, collate_fn=collate_batch)

In [27]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_rate):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional)
        self.fc = nn.Linear(hidden_dim*2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths=[embedded.size(1) for i in range(embedded.size(0))])

        _, (hidden, _) = self.lstm(packed_embedded)

        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))

1042